In [1]:
%%html
<style type="text/css">
  span.ecb { background: yellow; }
</style>

In [2]:
import numpy as np
from functools import partial
from scipy.spatial import cKDTree

# ATMO 5331 - Homework 5 & 6 - Fall 2023
## Due **Thursday** 30 Nov, 2023, 11:59 pm.
## *Worth two assignments*

When doing this homework, remember that you have two jobs:
1. Make it work.
2. Clean it up so that I can understand what you've done. If you think I might not understand, document it with a comment or a function docstring.

You should present your work with a clear logical progression. If that seems like a hassle, remember that in doing so you are practicing skills that are expected in your thesis and journal publications.

You may work alone or in pairs. I will not be adjudicating relative level of effort in group work, so you are responsible for ensuring that you and your partner contribute equally.

<span class="ecb">Comments by ECB</span>

**1.** Copy in your setup from HW3, so that you have the radar data, radar locations, and analysis grid available. Use only the tangent plane cartesian system part, and you don't need to include the plots. Take the time to clean up your original code so that it's the minimally necessary set of variables and functions.

**2.** Configuration of the weighting scheme requires that we know the typical data spacing. Following [TD2000](https://journals.ametsoc.org/view/journals/atot/17/2/1520-0426_2000_017_0105_rdoa_2_0_co_2.xml?tab_body=fulltext-display), define the data spacing as the distance betweent two radar gates at the maximum range. Use the point in your analysis grid that is farthest from the radar, and then find the maximum spacing in elevation angle at this range. Finally, calculate the difference in linear units between the two radar gates you idenified.

Please provide an answer to these two questions:
- What is the maximum distance from the radar in the objective analysis domain?
- What is the maximum spacing between two adjacent data points?

**3.** Below, the function `oban` (for "objective analysis") mimics the call signature of the MetPy `interpolate_to_points` function. Its principal difference is the `weight_func` argument, which takes a function instead of a string describing an interpolation method. 

The `oban` function passes `weight_func` only the distances, so it is necessary to use `partial` to pre-fill the function with any other arguments needed to configure the weight function. The `sample_weights` function below shows how this works.

For fun, I've also included a seasonal illustration of the use of `partial`.

For this question, your jobs are as follows.

**a.**  Specify a cutoff radius. Based on the last homework assignment, what is a good distance to use as a multiple of the data spacing? Make sure to adjust your set of input data points to include the necessary margin beyond the perimeter of the analysis grid.

**b.**  Implement a `barnes` function and then use it with `oban` to calculate an analysis for reflectivity on the target grid.  Note that you will need to complete the `oban` function in a way that will work with any weight function.

**c.**  Calculate a Barnes analysis using MetPy, as in the last assignment, and find the difference (yours - MetPy). They probably won't be the same, even for a sane configuration of parameters; that's ok.

**d.**  Plot the original data, the two analyses, and the difference.


In [3]:
from functools import partial

In [4]:
def thanksgiving(holiday_question, pie=False):
    print(holiday_question, "pie?", pie)

In [5]:
thanksgiving("On this day will I eat", pie=True)

On this day will I eat pie? True


In [6]:
bad_thanksgiving = partial(thanksgiving, "On this day I will eat")

In [7]:
bad_thanksgiving("No, because none were made for me.")

On this day I will eat pie? No, because none were made for me.


In [8]:
def oban(points, values, xi, weight_func, search_radius):
    """
    points: N,2 data point locations
    values: N data values
    xi: M,2 analysis locations
    weight_func is a function that accepts a single argument r that is the
        distance between the analysis location and all points within search_radius
        of the analysis location.
    """
    
    # Find all points in the vicinity of each analysis location in xi
    tree = cKDTree(points)
    query = tree.query_ball_point(xi, search_radius)
    
    analysis = np.zeros(xi.shape[0])
    
    # This is linear (times the typical neighborhood size) in the number of analysis points
    for i, (analysis_point, neighbors) in enumerate(zip(xi, query)):
        data = values[neighbors]
        data_locations = points[neighbors,:]
        # use data, data_locations, analysis_point, and weight_func to fill in the rest of the analysis
        analysis[i] = None # your code here
    return analysis

def sample_weights(r, value=None):
    return np.zeros_like(r) + value

my_weight_func = partial(sample_weights, value=3.0)
my_test_ranges = np.arange(10.0)
my_test_weights = my_weight_func(my_test_ranges) # oban will call my_weight_func like so
print(my_test_weights) 

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


**4.** Let's say we want to use another filter from [Harris (1978)](https://ieeexplore.ieee.org/iel5/5/31261/01455106.pdf?casa_token=nY_Vus-tiGQAAAAA:1K1Z17V0-r1wCpI7TlY0OFKZGTTtigj1xTtyLAj_DEkaAVYnkDVUh7Kl0BLFJjQZ4647zYZJm4c-) for our continuous data. Those functions are specified for discrete data, with a hard cutoff after $N$ samples. It would be logical to cut off all our analyses after the same cutoff radius for all data, so that our understanding of the filter function sidelobe behavior from discrete theory can be applied to continous data in an even-handed way.

So, let's repeat the previous question, but now using the Rectangular and Blackman-Harris weights. You will need to use Harris (1978) for the mathematical formulation of the windows, as defined below.

**a.**  Implement a `rect` function and then use it with `oban` to calculate an analysis on the target grid.

**b.**  Implement a `blackman_harris` function and then use it with `oban` to calculate an analysis on the target grid. Use the minimum 4-term Blackman-Harris formulation as in the `scipy.signal.blackmanharris` docs whose coefficients are the -92 dB 4-term window in the table on p. 65 of Harris.

**c.**  Include in this notebook, using a Markdown cell and the $\LaTeX$ functionality, a narrated derivation that shows how you converted the discrete, non-dimensional formulation of the Blackman-Harris weight function to a continuous, dimensional form.

**d.**  Make a plot of the weight functions as a function of distance from zero to your cutoff radius.

**e.**  Plot the original data and the two analyses.



**5.** Let's compare the different filters.

**a.** Plot the original data, and using the Barnes filter as a common point of comparison, plot the difference with the other two analyses you calculated.

**b.** Which weighting scheme preserves the greatest detail in fine-scale structure? Illustrate this by discussing a local minimum and a local maximum in the original data vs. the difference fields. Does it make sense in terms of the theoretical response functions we calcualted in the previous homework?